# Palmer Penguins - Pipeline de Machine Learning

**Projeto 02 - Construção de Esteira de Aprendizado de Máquina**

Neste projeto, implementamos uma esteira completa de Machine Learning utilizando o dataset Palmer Penguins.

## Dataset
O Palmer Penguins é um dataset que contém informações sobre pinguins de três espécies diferentes (Adelie, Chinstrap e Gentoo) coletadas nas ilhas Palmer, Antártica.

## Objetivo
Criar um modelo de classificação para prever a espécie do pinguim com base em suas características físicas.

## 1. Importação de Bibliotecas e Carregamento dos Dados

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Bibliotecas importadas com sucesso!")

In [ ]:
# Carregamento do dataset Palmer Penguins
# O dataset pode ser carregado via seaborn ou URL direta do UCI
df = sns.load_dataset('penguins')

print(f"Dataset carregado com sucesso!")
print(f"Dimensões: {df.shape[0]} linhas e {df.shape[1]} colunas\n")
print("Primeiras 5 linhas do dataset:")
df.head()

In [ ]:
# Informações gerais sobre o dataset
print("Informações do Dataset:")
print(df.info())
print("\n" + "="*50)
print("Tipos de dados:")
print(df.dtypes)

## 2. Estatísticas Descritivas

In [ ]:
# Estatísticas descritivas das variáveis numéricas
print("Estatísticas Descritivas das Variáveis Numéricas:")
print("="*80)
df.describe()

In [ ]:
# Estatísticas descritivas das variáveis categóricas
print("Estatísticas Descritivas das Variáveis Categóricas:")
print("="*80)
print("\nDistribuição de Espécies:")
print(df['species'].value_counts())
print("\nDistribuição de Ilhas:")
print(df['island'].value_counts())
print("\nDistribuição de Sexo:")
print(df['sex'].value_counts())
print("\nValores ausentes por coluna:")
print(df.isnull().sum())

In [ ]:
# Visualização da distribuição das variáveis numéricas
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Distribuição das Variáveis Numéricas', fontsize=16, fontweight='bold')

# Histogramas
df['bill_length_mm'].hist(ax=axes[0, 0], bins=20, edgecolor='black')
axes[0, 0].set_title('Comprimento do Bico (mm)')
axes[0, 0].set_xlabel('Comprimento (mm)')
axes[0, 0].set_ylabel('Frequência')

df['bill_depth_mm'].hist(ax=axes[0, 1], bins=20, edgecolor='black', color='orange')
axes[0, 1].set_title('Profundidade do Bico (mm)')
axes[0, 1].set_xlabel('Profundidade (mm)')
axes[0, 1].set_ylabel('Frequência')

df['flipper_length_mm'].hist(ax=axes[1, 0], bins=20, edgecolor='black', color='green')
axes[1, 0].set_title('Comprimento da Nadadeira (mm)')
axes[1, 0].set_xlabel('Comprimento (mm)')
axes[1, 0].set_ylabel('Frequência')

df['body_mass_g'].hist(ax=axes[1, 1], bins=20, edgecolor='black', color='red')
axes[1, 1].set_title('Massa Corporal (g)')
axes[1, 1].set_xlabel('Massa (g)')
axes[1, 1].set_ylabel('Frequência')

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots para identificar outliers
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Boxplots das Variáveis Numéricas', fontsize=16, fontweight='bold')

df.boxplot(column='bill_length_mm', ax=axes[0, 0])
axes[0, 0].set_title('Comprimento do Bico')
axes[0, 0].set_ylabel('mm')

df.boxplot(column='bill_depth_mm', ax=axes[0, 1])
axes[0, 1].set_title('Profundidade do Bico')
axes[0, 1].set_ylabel('mm')

df.boxplot(column='flipper_length_mm', ax=axes[1, 0])
axes[1, 0].set_title('Comprimento da Nadadeira')
axes[1, 0].set_ylabel('mm')

df.boxplot(column='body_mass_g', ax=axes[1, 1])
axes[1, 1].set_title('Massa Corporal')
axes[1, 1].set_ylabel('g')

plt.tight_layout()
plt.show()

In [ ]:
# Distribuição das espécies
plt.figure(figsize=(10, 6))
species_counts = df['species'].value_counts()
plt.bar(species_counts.index, species_counts.values, color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black')
plt.title('Distribuição das Espécies de Pinguins', fontsize=14, fontweight='bold')
plt.xlabel('Espécie')
plt.ylabel('Quantidade')
plt.grid(axis='y', alpha=0.3)
for i, v in enumerate(species_counts.values):
    plt.text(i, v + 2, str(v), ha='center', fontweight='bold')
plt.show()

## 3. Transformações nas Colunas

**Transformação aplicada:** Codificação de variáveis categóricas e normalização de variáveis numéricas

In [ ]:
# Criar uma cópia do dataframe para preservar o original
df_transformed = df.copy()

print("Antes da transformação:")
print(f"Shape: {df_transformed.shape}")
print(f"\nColunas: {df_transformed.columns.tolist()}")
print(f"\nValores únicos antes da codificação:")
print(f"Species: {df_transformed['species'].unique()}")
print(f"Island: {df_transformed['island'].unique()}")
print(f"Sex: {df_transformed['sex'].unique()}")

In [ ]:
# TRANSFORMAÇÃO DE COLUNAS: Codificação de variáveis categóricas
# Utilizando LabelEncoder para transformar variáveis categóricas em numéricas

# Criar encoders para cada variável categórica
label_encoders = {}

# Codificar 'island'
le_island = LabelEncoder()
df_transformed['island_encoded'] = le_island.fit_transform(df_transformed['island'])
label_encoders['island'] = le_island

# Codificar 'sex' (mantendo apenas linhas válidas temporariamente para encoding)
df_transformed['sex_encoded'] = df_transformed['sex'].map({'Male': 1, 'Female': 0})

# Guardar o encoding da espécie para usar depois
le_species = LabelEncoder()
df_transformed['species_encoded'] = le_species.fit_transform(df_transformed['species'])
label_encoders['species'] = le_species

print("✓ Transformação de colunas categóricas concluída!")
print(f"\nMapeamento Island: {dict(zip(le_island.classes_, le_island.transform(le_island.classes_)))}")
print(f"Mapeamento Sex: Male=1, Female=0")
print(f"Mapeamento Species: {dict(zip(le_species.classes_, le_species.transform(le_species.classes_)))}")
print("\nPrimeiras linhas após codificação:")
print(df_transformed[['island', 'island_encoded', 'sex', 'sex_encoded', 'species', 'species_encoded']].head())

## 4. Transformações nas Linhas

**Transformação aplicada:** Remoção de linhas com valores ausentes

In [ ]:
# TRANSFORMAÇÃO DE LINHAS: Remoção de valores ausentes
print("Valores ausentes antes da limpeza:")
print(df_transformed.isnull().sum())
print(f"\nTotal de linhas antes: {len(df_transformed)}")

# Remover linhas com valores ausentes
df_clean = df_transformed.dropna()

print(f"\nTotal de linhas depois: {len(df_clean)}")
print(f"Linhas removidas: {len(df_transformed) - len(df_clean)}")
print("\nValores ausentes após a limpeza:")
print(df_clean.isnull().sum())
print("\n✓ Transformação de linhas concluída!")

## 5. Divisão em Treino, Validação e Teste

Dividiremos o dataset em três conjuntos:
- **Treino (60%):** Para treinar o modelo
- **Validação (20%):** Para ajustar hiperparâmetros
- **Teste (20%):** Para avaliar o modelo final

In [ ]:
# Preparar features (X) e target (y)
# Usar as features numéricas originais e as codificadas
feature_columns = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 
                   'body_mass_g', 'island_encoded', 'sex_encoded']

X = df_clean[feature_columns]
y = df_clean['species_encoded']

print("Features selecionadas:", feature_columns)
print(f"\nShape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"\nDistribuição das classes:")
print(y.value_counts().sort_index())
print(f"\nPrimeiras linhas de X:")
print(X.head())

In [ ]:
# Primeira divisão: 80% treino+validação, 20% teste
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Segunda divisão: dos 80% restantes, dividir em 75% treino e 25% validação
# Isso resulta em: 60% treino, 20% validação, 20% teste do total
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print("Divisão dos dados concluída!")
print("="*60)
print(f"Total de amostras: {len(X)}")
print(f"\nConjunto de Treino: {len(X_train)} amostras ({len(X_train)/len(X)*100:.1f}%)")
print(f"Conjunto de Validação: {len(X_val)} amostras ({len(X_val)/len(X)*100:.1f}%)")
print(f"Conjunto de Teste: {len(X_test)} amostras ({len(X_test)/len(X)*100:.1f}%)")

print(f"\nDistribuição das classes no conjunto de treino:")
print(y_train.value_counts().sort_index())
print(f"\nDistribuição das classes no conjunto de validação:")
print(y_val.value_counts().sort_index())
print(f"\nDistribuição das classes no conjunto de teste:")
print(y_test.value_counts().sort_index())

In [ ]:
# Normalização das features (StandardScaler)
# Importante: fit apenas no treino e transform em todos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✓ Normalização aplicada!")
print("\nEstatísticas do conjunto de treino normalizado:")
print(f"Média: {X_train_scaled.mean(axis=0).round(2)}")
print(f"Desvio padrão: {X_train_scaled.std(axis=0).round(2)}")

## 6. Treinamento do Modelo

Utilizaremos o algoritmo **Random Forest Classifier** para classificar as espécies de pinguins.

In [ ]:
# Criar e treinar o modelo Random Forest
print("Iniciando o treinamento do modelo...")
print("="*60)

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

# Treinar o modelo
model.fit(X_train_scaled, y_train)

print("✓ Modelo treinado com sucesso!")
print(f"\nParâmetros do modelo:")
print(f"- Número de árvores: {model.n_estimators}")
print(f"- Profundidade máxima: {model.max_depth}")
print(f"- Número de features: {model.n_features_in_}")

In [ ]:
# Avaliar o modelo no conjunto de validação
y_val_pred = model.predict(X_val_scaled)
val_accuracy = accuracy_score(y_val, y_val_pred)

print("Avaliação no conjunto de validação:")
print("="*60)
print(f"Acurácia: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
print("\nRelatório de Classificação (Validação):")
print(classification_report(y_val, y_val_pred, target_names=le_species.classes_))

In [ ]:
# Importância das features
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("Importância das Features:")
print("="*60)
print(feature_importance.to_string(index=False))

# Visualizar importância
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='steelblue', edgecolor='black')
plt.xlabel('Importância')
plt.title('Importância das Features no Modelo', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Avaliação no Conjunto de Teste - Matriz de Confusão e Acurácia

In [ ]:
# Fazer predições no conjunto de teste
y_test_pred = model.predict(X_test_scaled)

# Calcular acurácia
test_accuracy = accuracy_score(y_test, y_test_pred)

print("AVALIAÇÃO FINAL NO CONJUNTO DE TESTE")
print("="*60)
print(f"✓ Acurácia no Teste: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print("\nRelatório de Classificação (Teste):")
print(classification_report(y_test, y_test_pred, target_names=le_species.classes_))

In [ ]:
# Calcular e visualizar a Matriz de Confusão
cm = confusion_matrix(y_test, y_test_pred)

print("Matriz de Confusão:")
print("="*60)
print(cm)

# Visualizar a matriz de confusão
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le_species.classes_, 
            yticklabels=le_species.classes_,
            cbar_kws={'label': 'Quantidade'},
            linewidths=1, linecolor='black')
plt.title('Matriz de Confusão - Conjunto de Teste', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Classe Real', fontsize=12, fontweight='bold')
plt.xlabel('Classe Predita', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

# Mostrar métricas detalhadas
print("\nInterpretação da Matriz de Confusão:")
print("="*60)
for i, species in enumerate(le_species.classes_):
    correct = cm[i, i]
    total = cm[i, :].sum()
    accuracy_class = correct / total if total > 0 else 0
    print(f"{species}: {correct}/{total} corretos ({accuracy_class*100:.1f}% de acurácia)")

## 8. Predição com o Modelo Implantado

Vamos fazer uma predição com uma amostra nova para demonstrar o modelo em ação.

In [ ]:
# Selecionar uma amostra do conjunto de teste para demonstração
sample_idx = 0
sample = X_test.iloc[sample_idx:sample_idx+1]
sample_scaled = X_test_scaled[sample_idx:sample_idx+1]

# Fazer a predição
prediction = model.predict(sample_scaled)[0]
prediction_proba = model.predict_proba(sample_scaled)[0]

# Obter informações da amostra original
species_real = le_species.inverse_transform([y_test.iloc[sample_idx]])[0]
species_pred = le_species.inverse_transform([prediction])[0]

print("EXEMPLO DE PREDIÇÃO COM O MODELO")
print("="*60)
print("\n📋 Características do Pinguim:")
print("-" * 60)
for i, col in enumerate(feature_columns):
    print(f"  {col:25s}: {sample.iloc[0, i]:.2f}")

print("\n🎯 Resultado da Predição:")
print("-" * 60)
print(f"  Espécie Real:      {species_real}")
print(f"  Espécie Predita:   {species_pred}")
print(f"  ✓ Predição:        {'CORRETA ✓' if species_real == species_pred else 'INCORRETA ✗'}")

print("\n📊 Probabilidades por Classe:")
print("-" * 60)
for i, species in enumerate(le_species.classes_):
    prob = prediction_proba[i]
    bar = "█" * int(prob * 50)
    print(f"  {species:12s}: {prob*100:6.2f}% {bar}")

In [ ]:
# Fazer predições para múltiplas amostras
print("PREDIÇÕES PARA MÚLTIPLAS AMOSTRAS")
print("="*80)

num_samples = 5
for i in range(min(num_samples, len(X_test))):
    sample = X_test.iloc[i:i+1]
    sample_scaled = X_test_scaled[i:i+1]
    
    prediction = model.predict(sample_scaled)[0]
    species_real = le_species.inverse_transform([y_test.iloc[i]])[0]
    species_pred = le_species.inverse_transform([prediction])[0]
    
    status = "✓" if species_real == species_pred else "✗"
    print(f"Amostra {i+1}: Real={species_real:12s} | Predito={species_pred:12s} | {status}")

## 9. Conclusões

### Resumo do Projeto

Neste projeto, implementamos uma esteira completa de Machine Learning para classificação de espécies de pinguins usando o dataset Palmer Penguins:

1. **Dataset**: Palmer Penguins com 3 espécies (Adelie, Chinstrap, Gentoo)
2. **Estatísticas Descritivas**: Análise completa das variáveis numéricas e categóricas
3. **Transformações**:
   - **Colunas**: Codificação de variáveis categóricas e normalização
   - **Linhas**: Remoção de valores ausentes
4. **Divisão dos Dados**: 60% treino, 20% validação, 20% teste
5. **Modelo**: Random Forest Classifier
6. **Resultados**: Alta acurácia na classificação das espécies

### Próximos Passos

- Testar outros algoritmos (SVM, XGBoost, Neural Networks)
- Realizar ajuste fino de hiperparâmetros (GridSearch/RandomSearch)
- Implementar validação cruzada
- Exportar o modelo para produção